In [1]:
import pandas as pd
import datetime
import subprocess
import os
from pathlib import Path

# Process *les journaux officiels*

In [2]:
tmpfile = "tmp.txt"
df_file = "data_processed.csv"
if os.path.isfile(df_file):
    df = pd.read_csv(df_file)
    jo_list = sorted(df["jo"].unique())
else:
    df = pd.DataFrame(columns=["numero", "série", "sexe", "age", "dep", "jo", "delai"])
    jo_list = []

for decret in Path(".").glob("decrets/*.pdf"):
    # Skip if current JO is already treated
    jo = str(decret).partition("joe_")[2].partition("_")[0]
    if int(jo) in jo_list:
        continue
    
    print(f"Process JO {jo}...")
    date_jo = datetime.datetime.strptime(jo, "%Y%m%d")
    with open(tmpfile, "w") as f:
        subprocess.check_call(["pdf2txt.py", decret], stdout=f)
#         subprocess.check_call(["python", "D:/Programs/Miniconda3/Scripts/pdf2txt.py", str(decret)], stdout=f)
    
    with open(tmpfile) as f:
        content = f.read()
        keys = ["\n\n", ", \n"]
        for key in keys:
            content = content.replace(key, "\n")
        content = content.replace("\ndép", ", dép")
        content = content.replace("dép. \n", ", dép. ")
        content = content.replace("X \n", "X")
        content = content.replace(". \n", "^")
        content = content.replace(": \n", "^")
        content = content.replace("\n", "")
        content = content.replace("^", ".\n")
        lines = content.split("\n")
    
    for line in lines:
        if "NAT," not in line:
            continue
        else:
            if "née" in line:
                sexe = "F"
            else:
                sexe = "H"
            line = line.replace(" au ", " à ").replace(" aux ", " à ")
            naissance = line.partition(" le ")[2].partition(" à ")[0].replace(" ", "")
            try:
                date_naissance = datetime.datetime.strptime(naissance, "%d/%m/%Y")
            except ValueError:
                continue
            age = (date_jo - date_naissance).days / 365.2425
            dep = line.partition("dép.")[2].partition(",")[0].replace(" ", "")
            if len(dep) == 0:
                continue
            elif "Dt" in dep:
                dep = dep.partition("Dt")[0]
            numero = line.partition("NAT,")[2].partition(",")[0].replace(" ", "")
            serie = numero[:8]
            year = int(serie.partition("X")[0])
            weeknum = int(serie.partition("X")[2][:3]) + 1
            if weeknum > 53:
                continue
            else:
                date_serie = datetime.datetime.strptime(f"{year} {weeknum}-3", "%G %V-%w")
                delai = (date_jo - date_serie).days / (365.2425 / 12)
            line = {"numero": numero, "serie": serie, "sexe": sexe, "age": age, "dep": dep, "jo": jo, "delai": delai}
            df = df.append(line, ignore_index=True)

os.remove(tmpfile)

Process JO 20190929...


In [3]:
df.tail()

,numero,serie,sexe,age,dep,jo,delai
23773,2019X007565,2019X007,H,47.617679,027,20190929,7.260929
23774,2019X006074,2019X006,H,21.128428,092,20190929,7.490914
23775,2019X008716,2019X008,F,27.995099,036,20190929,7.030945
23776,2018X044255,2018X044,H,37.032930,006,20190929,10.710692
23777,2019X009158,2019X009,F,34.284072,095,20190929,6.800961


In [4]:
df.isnull().values.any()

False

In [5]:
df.to_csv("data_processed.csv", index=False)